##### Copyright 2021 The TensorFlow Authors.

### Install the TFLite Support Library

In [26]:
!pip install -q tflite_support

2197.12s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


### Install the TensorFlow Object Detection API


In [3]:
import os
import pathlib


In [5]:
cd /home/poornimadevikr/git/mymodels/models/research

/home/poornimadevikr/git/mymodels/models/research


In [6]:
# Compile the protos
!protoc object_detection/protos/*.proto --python_out=.

In [7]:
# Copy the setup.py from the object_detection folder to models/research folder
!cp /home/poornimadevikr/git/mymodels/models/research/object_detection/packages/tf2/setup.py /home/poornimadevikr/git/mymodels/models/research/


In [8]:
pip install -q .

Note: you may need to restart the kernel to use updated packages.


### Import the necessary libraries

In [9]:
import matplotlib
import matplotlib.pyplot as plt

import os
import random
import io
import imageio
import glob
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import config_util
from object_detection.builders import model_builder

%matplotlib inline

2025-04-08 10:13:36.908315: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-08 10:13:37.832750: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2025-04-08 10:13:37.832865: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2025-04-08 10:13:37.832878: W tensorfl

## Generate TensorFlow Lite Model

### Step 1: Export TFLite inference graph

First, we invoke `export_tflite_graph_tf2.py` to generate a TFLite-friendly intermediate SavedModel. This will then be passed to the TensorFlow Lite Converter for generating the final model.

Use `--help` with the above script to get the full list of supported parameters.
These can fine-tune accuracy and speed for your model.

In [ ]:
!python /home/poornimadevikr/git/mymodels/models/research/object_detection/export_tflite_graph_tf2.py \
    --trained_checkpoint_dir {'/home/poornimadevikr/git/mymodels/models/trainingcheck'} \
    --output_directory {'/home/poornimadevikr/git/mymodels/models/research/object_detection/my_notebooks/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'} \
    --pipeline_config_path {'/home/poornimadevikr/git/mymodels/models/research/object_detection/my_notebooks/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'}

140.70s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


2025-04-08 10:15:02.449564: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-08 10:15:03.493063: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2025-04-08 10:15:03.493187: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2025-04-08 10:15:03.493202: W tensorfl

### Step 2: Convert to TFLite

Use the [TensorFlow Lite Converter](https://www.tensorflow.org/lite/convert) to
convert the `SavedModel` to TFLite. Note that you need to use `from_saved_model`
for TFLite conversion with the Python API.

You can also leverage
[Post-training Quantization](https://www.tensorflow.org/lite/performance/post_training_quantization)
to
[optimize performance](https://www.tensorflow.org/lite/performance/model_optimization)
and obtain a smaller model. here, we use the [dynamic range quantization](https://www.tensorflow.org/lite/performance/post_training_quant).

In [14]:
_TFLITE_MODEL_PATH = "/home/poornimadevikr/git/mymodels/models/research/tflite/model.tflite"

converter = tf.lite.TFLiteConverter.from_saved_model('/home/poornimadevikr/git/mymodels/models/research/tflite/saved_model')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with open(_TFLITE_MODEL_PATH, 'wb') as f:
  f.write(tflite_model)

2025-04-08 10:30:13.354249: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2025-04-08 10:30:13.354379: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2025-04-08 10:30:13.354617: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /home/poornimadevikr/git/mymodels/models/research/tflite/saved_model
2025-04-08 10:30:13.403435: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2025-04-08 10:30:13.403577: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /home/poornimadevikr/git/mymodels/models/research/tflite/saved_model
2025-04-08 10:30:13.567018: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2025-04-08 10:30:14.015096: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: /home/poornimadevikr/git/mymodels/models/research/tfl